In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
# imports

import time
import os
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torchvision.utils import draw_bounding_boxes
from torchvision.transforms import ToPILImage

from engine import get_detection_model, train_one_epoch, evaluate, train
from dataset import TrashCanDataset
from transforms import Compose, ToTensor
from utils import collate_fn, show_sample

In [2]:
root_path = "/Users/user/Desktop/dataset"
save_path = "/Users/user/Desktop/dataset"
show_example = True
num_epochs = 10
batch_size = 1

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [4]:
# Create Dataloader
dataset = TrashCanDataset(root_path, transforms=Compose([ToTensor()]))

# split the dataset in train and test set
torch.manual_seed(42)
indices = torch.randperm(len(dataset)).tolist()
dataset_train = torch.utils.data.Subset(dataset, indices[:-50])
dataset_test = torch.utils.data.Subset(dataset, indices[-50:])

train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

# Create the model
model = get_detection_model(num_classes=23)
model.to(device)


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [5]:
if show_example:
  img, target = dataset[10]
  show_sample(img, target)

In [6]:
# Construct optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

In [7]:
# Train the model
train(model, num_epochs, train_loader, test_loader, optimizer, device, save_path)

epoch 0/10..
Loss value after 10 batches is 1.08
Loss value after 20 batches is 0.82
Loss value after 30 batches is 0.76
Loss value after 40 batches is 0.68
Loss value after 50 batches is 0.62
Loss value after 60 batches is 0.61
Loss value after 70 batches is 0.6
Loss value after 80 batches is 0.59
Loss value after 90 batches is 0.57
Loss value after 100 batches is 0.57
Loss value after 110 batches is 0.56


KeyboardInterrupt: 

In [ ]:
img = torch.tensor(test_image[0]*255, dtype=torch.uint8)
box = prediction[0]['boxes'].tolist()
for i in range(len(box)):
    box[i] = [int(x) for x in box[i]]
box = torch.tensor(box, dtype=torch.int)
img = draw_bounding_boxes(img, box, width=5, fill=True)
img = ToPILImage()(img)
#img.show()
plt.imshow(img)